In [1]:
from imutils.video import VideoStream
from imutils.video import FPS
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import imutils
import time
import cv2
import pytesseract

In [2]:
def decode_predictions(scores, geometry):
	(numRows, numCols) = scores.shape[2:4]
	rects = []
	confidences = []

	for y in range(0, numRows):
		scoresData = scores[0, 0, y]
		xData0 = geometry[0, 0, y]
		xData1 = geometry[0, 1, y]
		xData2 = geometry[0, 2, y]
		xData3 = geometry[0, 3, y]
		anglesData = geometry[0, 4, y]

		for x in range(0, numCols):
			if scoresData[x] < args["min_confidence"]:
				continue

			(offsetX, offsetY) = (x * 4.0, y * 4.0)

			angle = anglesData[x]
			cos = np.cos(angle)
			sin = np.sin(angle)

			h = xData0[x] + xData2[x]
			w = xData1[x] + xData3[x]

			endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
			endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
			startX = int(endX - w)
			startY = int(endY - h)

			rects.append((startX, startY, endX, endY))
			confidences.append(scoresData[x])

	return (rects, confidences)

In [3]:
args = {"video":"board.mp4", "east":"east_text_detection.pb","min_confidence":0.5,"width":320,"height":320}

In [4]:
(W, H) = (None, None)
(newW, newH) = (args["width"], args["height"])
(rW, rH) = (None, None)

In [5]:
layerNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(args["east"])

[INFO] loading EAST text detector...


In [6]:
if not args.get("video", False):
	print("[INFO] starting video stream...")
	vs = VideoStream(src=0).start()
	time.sleep(1.0)
else:
	vs = cv2.VideoCapture(args["video"])
fps = FPS().start()

In [ ]:
while True:
	frame = vs.read()
	frame = frame[1] if args.get("video", False) else frame

	if frame is None:
		break
	frame = imutils.resize(frame, width=1000)
	orig = frame.copy()
	if W is None or H is None:
		(H, W) = frame.shape[:2]
		rW = W / float(newW)
		rH = H / float(newH)
	frame = cv2.resize(frame, (newW, newH))
    
	blob = cv2.dnn.blobFromImage(frame, 1.0, (newW, newH),(123.68, 116.78, 103.94), swapRB=True, crop=False)
    
	net.setInput(blob)
	(scores, geometry) = net.forward(layerNames)
    
	(rects, confidences) = decode_predictions(scores, geometry)
	boxes = non_max_suppression(np.array(rects), probs=confidences)
	results = []
    
	for (startX, startY, endX, endY) in boxes:
		startX = int(startX * rW)
		startY = int(startY * rH)
		endX = int(endX * rW)
		endY = int(endY * rH)
		cv2.rectangle(orig, (startX, startY), (endX, endY),(0, 0, 255), 2)
		r = orig[startY:endY, startX:endX]
	fps.update()
	cv2.imshow("Broad Detection", orig)
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
		break

In [ ]:
if not args.get("video", False):
	vs.stop()
else:
	vs.release()
cv2.destroyAllWindows()